In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution3D,Conv3D,Add,Concatenate, MaxPool3D,LeakyReLU,add, Convolution2D,BatchNormalization,AveragePooling3D, GlobalAveragePooling3D, ZeroPadding3D
from keras import backend as K
import cv2
import operator
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
import nibabel as nib
import pandas as pd
from scipy.ndimage import rotate
import random
import nibabel as nib
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np
import nibabel as nib
from matplotlib import animation, rc
from scipy.ndimage import rotate
import glob
import os
from sklearn.model_selection import KFold, StratifiedKFold,train_test_split
from scipy.ndimage import zoom
from matplotlib.patches import PathPatch, Rectangle
from IPython.display import HTML
from tensorflow.keras import layers
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

print("Using GPU:", gpus)



# lidc load

In [ ]:
luna_df = pd.read_excel('/home/m-health/TBM/Lung/final_Luna16.xlsx')
column_luna = 'malignancy'
luna_series=luna_df['Series Uid'].tolist()

luna_folder = '/home/m-health/TBM/Lung/Luna_16_hu_from_crop_64'  
luna = []
for i in luna_series:
    nii_file_path = os.path.join(luna_folder, f'{i}.nii')  # Assuming NII files have the extension '.nii'
    
    if os.path.exists(nii_file_path):
        luna.append(i)
luna=np.array(luna)

print(luna.shape)





# Specify the path to the Excel file
file_path = '/home/m-health/TBM/Lung/test_initialtrain_vai_set.xlsx'
df = pd.read_excel(file_path)

z_train = df.iloc[:, 2]  

z_test_fixed = df.iloc[:, 0]  
nan_count = z_test_fixed.isna().sum()
z_test = z_test_fixed.dropna()

z_val_fixed = df.iloc[:, 1]  
nan_count_val = z_val_fixed.isna().sum()
z_val = z_val_fixed.dropna()


function for rotation

In [ ]:
BS= 8 # batch size
E = 200 #epoch

In [ ]:
def aug_rotate(x):

    rotated_slices = []
    for slice_idx in range(x.shape[2]):
        slice_data = x[:, :, slice_idx]  # Extract a single slice
        rotated_slice = rotate(slice_data, angle=angles[j], reshape=False, mode='nearest')
        rotated_slices.append(rotated_slice)

    # Create a new NIfTI image from the rotated slices
    rotated_img_data = np.stack(rotated_slices, axis=-1)
    # Create a new NIfTI image using the header information from the original image
    rotated_nifti = nib.Nifti1Image(rotated_img_data, img.affine)
    
    return rotated_nifti

split

In [ ]:
angles = [0, 45, 90, 135, 180, 225, 270, 315]



datas_test = []
labels_test = []
scans_test = []

datas_train = []
labels_train = []
scans_train = []

datas_val = []
labels_val = []
scans_val = []


for i in z_train:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join(luna_folder, f"{i}.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        for j in range(len(angles)):
            rotated_nifti = aug_rotate(data)
            datas_train.append(rotated_nifti.get_fdata())
            labels_train.append(lab)
                    
for i in z_val:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join(luna_folder, f"{i}.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_val.append(data)
        labels_val.append(lab)

for i in z_test:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join(luna_folder, f"{i}.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_test.append(data)
        labels_test.append(lab)
            
X_train = np.array(datas_train)
y_train = np.array(labels_train)
X_test = np.array(datas_test)
y_test = np.array(labels_test)
X_val = np.array(datas_val)
y_val = np.array(labels_val)


X_train = np.array(datas_train)
y_train = np.array(labels_train)
X_test = np.array(datas_test)
y_test = np.array(labels_test)
X_val = np.array(datas_val)
y_val = np.array(labels_val)

print(z_train.shape, z_test.shape, z_val.shape)
print(X_train.shape, X_test.shape, X_val.shape)

model train

### train with LCCNet

In [ ]:
def lcc():
    model = Sequential()
    model.add(Conv3D(32,(3,3,3), activation='relu', kernel_initializer='he_uniform', padding='same',input_shape=(48,48,48,1)))
    model.add(BatchNormalization())
    model.add(Conv3D(32, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    #model.add(Dropout(0.1))
    
    model.add(Conv3D(64, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(64, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    #model.add(Dropout(0.2))
    
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    model.add(Dropout(0.3))
    
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))

    model.add(Conv3D(256, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(256, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    #model.add(MaxPool3D((2, 2,2))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    #model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(8, activation='relu', kernel_initializer='he_uniform'))


    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_uniform'));
    return model

# for patch size 48* 48 *48

In [ ]:
save_dir = "/home/m-health/TBM/Lung/"

# ------------------------------------------------------------
strategy = tf.distribute.get_strategy()

# ------------------------------------------------------------
# 3) Build & compile model
# ------------------------------------------------------------
with strategy.scope():
    model = lcc()
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

# ------------------------------------------------------------
# 4) Callbacks
# ------------------------------------------------------------
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.001,
    patience=5,
    min_lr=1e-6
)

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(save_dir, "lcc_net.h5"),
    monitor='val_accuracy',
    save_best_only=True
)


# ------------------------------------------------------------
# 6) Train
# ------------------------------------------------------------

BS = BATCH_SIZE

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=E,
    batch_size=BATCH_SIZE,
    callbacks=[lr_callback, checkpoint_callback],
    verbose=1
)

# ------------------------------------------------------------
# 7) Load best model & evaluate
# ------------------------------------------------------------
best_model = tf.keras.models.load_model(os.path.join(save_dir, "lcc_net.h5"))
loss, accuracy = best_model.evaluate(X_test, y_test)

# ------------------------------------------------------------
# 8) Predictions
# ------------------------------------------------------------
predictions = best_model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int)

class_names = ['benign', 'malignant']
print(classification_report(y_test, predicted_classes, target_names=class_names, digits=5))

auc = roc_auc_score(y_test, predictions)
print("AUC:", auc)

# ------------------------------------------------------------
# 9) Confusion Matrix
# ------------------------------------------------------------
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Loss: {loss*100:.2f}')


# for patch size 64* 64 *64

In [ ]:
def lcc():
    model = Sequential()
    model.add(Conv3D(32,(3,3,3), activation='relu', kernel_initializer='he_uniform', padding='same',input_shape=(64,64,64,1)))
    model.add(BatchNormalization())
    model.add(Conv3D(32, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    #model.add(Dropout(0.1))
    
    model.add(Conv3D(64, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(64, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    #model.add(Dropout(0.2))
    
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    model.add(Dropout(0.3))
    
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))

    model.add(Conv3D(256, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(256, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    #model.add(MaxPool3D((2, 2,2))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    #model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(8, activation='relu', kernel_initializer='he_uniform'))


    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_uniform'));
    return model

In [ ]:
save_dir = "/home/m-health/TBM/Lung/"

# ------------------------------------------------------------
strategy = tf.distribute.get_strategy()

# ------------------------------------------------------------
# 3) Build & compile model
# ------------------------------------------------------------
with strategy.scope():
    model = lcc()
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

# ------------------------------------------------------------
# 4) Callbacks
# ------------------------------------------------------------
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.001,
    patience=5,
    min_lr=1e-6
)

checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(save_dir, "lcc_net_64.h5"),
    monitor='val_accuracy',
    save_best_only=True
)


# ------------------------------------------------------------
# 6) Train
# ------------------------------------------------------------

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=E,
    batch_size=BS,
    callbacks=[lr_callback, checkpoint_callback],
    verbose=1
)

# ------------------------------------------------------------
# 7) Load best model & evaluate
# ------------------------------------------------------------
best_model = tf.keras.models.load_model(os.path.join(save_dir, "lcc_net_64.h5"))
loss, accuracy = best_model.evaluate(X_test, y_test)

# ------------------------------------------------------------
# 8) Predictions
# ------------------------------------------------------------
predictions = best_model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int)

class_names = ['benign', 'malignant']
print(classification_report(y_test, predicted_classes, target_names=class_names, digits=5))

auc = roc_auc_score(y_test, predictions)
print("AUC:", auc)

# ------------------------------------------------------------
# 9) Confusion Matrix
# ------------------------------------------------------------
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Loss: {loss*100:.2f}')


In [ ]:
# # ------------------------------------------------------------
# # 10) Plot & Save Misclassified 3D Volumes (Show Middle Slice)
# # ------------------------------------------------------------
# import matplotlib.pyplot as plt
# import os
# import numpy as np

# wrong_pred_dir = os.path.join(save_dir, "wrong_predictions")
# os.makedirs(wrong_pred_dir, exist_ok=True)

# wrong_indices = np.where(predicted_classes.flatten() != y_test.flatten())[0]
# print(f"Total misclassified samples: {len(wrong_indices)}")

# for idx in wrong_indices:
#     vol = X_test[idx]  # shape: (64,64,64) or (64,64,64,1)

#     # ---- Handle channel dimension if present ----
#     if vol.ndim == 4 and vol.shape[-1] == 1:
#         vol = vol[..., 0]  # make it (64,64,64)

#     # ---- Extract the central slice ----
#     mid_slice = vol[:, :, vol.shape[2] // 2]

#     true_label = y_test[idx]
#     pred_label = predicted_classes[idx][0]
#     pred_prob = predictions[idx][0]

#     plt.figure(figsize=(4,4))
#     plt.imshow(mid_slice, cmap='gray')
#     plt.title(
#         f"True: {class_names[int(true_label)]}\n"
#         f"Pred: {class_names[int(pred_label)]} (p={pred_prob:.3f})"
#     )
#     plt.axis('off')

#     save_path = os.path.join(wrong_pred_dir, f"wrong_{idx}.png")
#     plt.savefig(save_path, bbox_inches='tight', dpi=150)
#     plt.close()

# print(f"Saved misclassified CT slice images to: {wrong_pred_dir}")


In [ ]:
# ------------------------------------------------------------
# # Plot & Save Correctly Classified 3D Volumes (Middle Slice)
# # ------------------------------------------------------------
# import os
# import numpy as np
# import matplotlib.pyplot as plt

# correct_pred_dir = os.path.join(save_dir, "correct_predictions")
# os.makedirs(correct_pred_dir, exist_ok=True)

# # Option: limit how many to save (set to None to save all)
# MAX_SAVE = None  # e.g., 200 or None

# # Find indices where prediction == ground truth
# correct_indices = np.where(predicted_classes.flatten() == y_test.flatten())[0]
# print(f"Total correctly-classified samples: {len(correct_indices)}")

# saved = 0
# for i, idx in enumerate(correct_indices):
#     if (MAX_SAVE is not None) and (saved >= MAX_SAVE):
#         break

#     vol = X_test[idx]  # e.g., (64,64,64) or (64,64,64,1)

#     # Remove singleton channel if present
#     if vol.ndim == 4 and vol.shape[-1] == 1:
#         vol = vol[..., 0]

#     # Defensive: ensure vol is 3D
#     if vol.ndim != 3:
#         # try to squeeze otherwise skip
#         vol = np.squeeze(vol)
#         if vol.ndim != 3:
#             print(f"Skipping index {idx}: unexpected shape {X_test[idx].shape}")
#             continue

#     # Extract middle axial slice
#     mid_slice = vol[:, :, vol.shape[2] // 2]

#     true_label = int(y_test[idx])
#     pred_label = int(predicted_classes[idx].flatten()[0])
#     pred_prob = float(predictions[idx].flatten()[0])

#     plt.figure(figsize=(4,4))
#     plt.imshow(mid_slice, cmap="gray")
#     plt.title(
#         f"True: {class_names[true_label]}\n"
#         f"Pred: {class_names[pred_label]} (p={pred_prob:.3f})",
#         fontsize=10
#     )
#     plt.axis('off')

#     fname = f"correct_idx{idx}_true-{class_names[true_label]}_pred-{class_names[pred_label]}_p{pred_prob:.3f}.png"
#     save_path = os.path.join(correct_pred_dir, fname)
#     plt.savefig(save_path, bbox_inches="tight", dpi=150)
#     plt.close()

#     saved += 1

# print(f"Saved {saved} correctly-classified sample plots to: {correct_pred_dir}")
